In [83]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2
from sklearn.model_selection import ParameterGrid


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Reading Image

In [84]:
image=cv2.imread('../images/2.png')
cv2.imshow('image',image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [85]:
error_image=False

## Image Pre-Processing

In [86]:
#! threshold the image
grey_image=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
_, thresholded_image = cv2.threshold(grey_image, 180, 255, cv2.THRESH_BINARY)  

cv2.imshow('Thresholded Image', thresholded_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [87]:
#! filter contours
image2=image.copy()
contours, _ = cv2.findContours(thresholded_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
if not contours:
    error_image=True
    print('No contours found')
filtered_contours = [contour for contour in contours if cv2.contourArea(contour) > 10000]  # Adjust the area threshold as needed

cv2.drawContours(image2, filtered_contours, -1, (0, 255, 0), 3)


cv2.imshow('grey_image',image2)
cv2.waitKey(0)
cv2.destroyAllWindows()

No contours found


In [88]:
#! fix prespective through the most fitting polygon wrapping
# the matrix is constant for sticker detection

height, width, _ = image.shape
corrected_image =np.zeros_like(grey_image)
if not error_image:
    contours = sorted(contours, key=cv2.contourArea, reverse=True)
    largest_contour = contours[0]
    epsilon = 0.02 * cv2.arcLength(largest_contour, True)
    approx_polygon = cv2.approxPolyDP(largest_contour, epsilon, True)

    corners = np.squeeze(approx_polygon)
    output_corners = np.array([[0, 0], [0, height], [width,height], [width, 0]], dtype=np.float32)  
    matrix = cv2.getPerspectiveTransform(corners.astype(np.float32), output_corners)
    corrected_image = cv2.warpPerspective(image, matrix, (width, height))  
    

#cv2.drawContours(image, [approx_polygon], -1, (0, 255, 0), 3)
cv2.imshow('grey_image',corrected_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [89]:
#! refix thresholds
mask = np.zeros_like(grey_image)
if not error_image:
    img_fix=corrected_image.copy()
    grey_image=cv2.cvtColor(img_fix,cv2.COLOR_BGR2GRAY)
    _, thresholded_image = cv2.threshold(grey_image, 180, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresholded_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    if  contours:
        contours = sorted(contours, key=cv2.contourArea, reverse=True)
        contours.pop(0)
        filtered_contours = [contour for contour in contours if cv2.contourArea(contour) > 10000]
        cv2.drawContours(img_fix, filtered_contours, -1, (0, 255, 0), 3)

        cv2.fillPoly(mask, filtered_contours, 255)
        mask = cv2.bitwise_not(mask)
    else: 
        error_image=True
cv2.imshow('Thresholded Image', mask)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Detect Straight Lines

In [90]:
#! TODO: Thresholds to be adapted
mask1=mask.copy()

#! thin the image
kernel = np.ones((3,3), np.uint8)
dilated = cv2.dilate(mask1, kernel, iterations=5)

#! get the horizontal lines
adaptive_horizontal_kernel = np.zeros((5, 5), dtype=np.uint8)
adaptive_horizontal_kernel[2, :] = 1
hit_or_miss_horizontal = cv2.dilate(dilated, adaptive_horizontal_kernel, iterations=17)
contours, _ = cv2.findContours(hit_or_miss_horizontal, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
if not error_image:
    contours = sorted(contours, key=cv2.contourArea, reverse=True)
    if contours:
       contours.pop(0)
    cv2.drawContours(hit_or_miss_horizontal, contours, -1, (0, 255, 0), 3)
    h_coordinates = []
    for contour in contours:
        # Get the bounding box of the contour
        x, y, w, h = cv2.boundingRect(contour)
        if(w > 100):
            h_coordinates.append((x,x+w,y,y+h))

    for x1,x2,y,h in h_coordinates:
        cv2.rectangle(mask1, (x1, y-30), (x2, y+30), (255, 255, 255), -1)
        
    #! get the first diagonal lines
    diagonal_kernel1 = np.array([[0, 0, 1],
                                [0, 1, 0],
                                [1, 0, 0]], dtype=np.uint8)
    hit_or_miss_diagonal1 = cv2.dilate(dilated, diagonal_kernel1, iterations=17)
    contours, _ = cv2.findContours(hit_or_miss_diagonal1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)
    if contours:
       contours.pop(0)
    #cv2.drawContours(hit_or_miss_diagonal1, contours, -1, (0, 255, 0), 3)
    rotated_rectangles = []

    for contour in contours:
        # Get the rotated rectangle that encloses the contour
        rotated_rect = cv2.minAreaRect(contour)
        
        rotated_rectangles.append(rotated_rect)

    for rotated_rect in rotated_rectangles:
        # Get the vertices of the rotated rectangle
        vertices = cv2.boxPoints(rotated_rect).astype(np.int0)
        area = cv2.contourArea(vertices)   
        # Draw the rotated rectangle on the original imagen after thresholding on area
        if area > 1000:
            cv2.drawContours(mask1, [vertices], 0, (255, 255, 255), 25)

    #! get the second diagonal lines
    diagonal_kernel2 = np.array([[1, 0, 0],
                                [0, 1, 0],
                                [0, 0, 1]], dtype=np.uint8)
    hit_or_miss_diagonal2 = cv2.dilate(dilated, diagonal_kernel2, iterations=17)
    contours, _ = cv2.findContours(hit_or_miss_diagonal2, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)
    if contours:
       contours.pop(0)
    #cv2.drawContours(hit_or_miss_diagonal1, contours, -1, (0, 255, 0), 3)
    rotated_rectangles = []

    for contour in contours:
        # Get the rotated rectangle that encloses the contour
        rotated_rect = cv2.minAreaRect(contour)
        
        rotated_rectangles.append(rotated_rect)

    for rotated_rect in rotated_rectangles:
        # Get the vertices of the rotated rectangle
        vertices = cv2.boxPoints(rotated_rect).astype(np.int0)
        area = cv2.contourArea(vertices)   
        # Draw the rotated rectangle on the original imagen after thresholding on area
        if area > 1000:
            cv2.drawContours(mask1, [vertices], 0, (255, 255, 255), 25)
            
    mask1=cv2.bitwise_not(mask1)
result = cv2.bitwise_or(mask, mask1)
cv2.imshow('Result Image', result)
cv2.waitKey(0)
cv2.destroyAllWindows()